# Required Modules Loading and Configs

In [1]:
import sys
import os
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import ConvergenceWarning
from scipy.stats import pearsonr
from itertools import combinations_with_replacement
from inc import *

%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 4)

# Dataset Load

In [2]:
data = pd.read_csv('dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index, format='%Y-%m-%d')
index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
data.set_index(index, inplace=True)
data[['Close', 'Open', 'High', 'Low']] = data[['Close', 'Open', 'High', 'Low']].replace(',', '', regex=True).astype(float)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1768 entries, 2013-03-01 to 2018-01-01
Freq: D
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Close                               1768 non-null   float64
 1   Open                                1768 non-null   float64
 2   High                                1768 non-null   float64
 3   Low                                 1768 non-null   float64
 4   Bitcoin Total Transaction Fees USD  1768 non-null   float64
 5   Bitcoin USD Exchange Trade Volume   1768 non-null   float64
 6   Bitcoin Hash Rate                   1768 non-null   float64
 7   Bitcoin Cost Per Transaction        1768 non-null   float64
 8   Bitcoin Number of Transactions      1768 non-null   float64
 9   Cocoa                               1768 non-null   float64
 10  Crude Oil                           1768 non-null   float64
 11  S&P500 Future    

# Feature Engineering

In [4]:
data['Direction'] = (data['Close'].diff() >= 0).astype(int)

In [5]:
lag_features = (
    'Close',
    'Open',
    'High',
    'Low',
    'Direction',
    'Bitcoin Total Transaction Fees USD',
    'Bitcoin USD Exchange Trade Volume',
    'Bitcoin Hash Rate',
    'Bitcoin Cost Per Transaction',
    'Bitcoin Number of Transactions'
)

data_with_lags = calc_lags(data, lag_features)

In [6]:
wma_features = (
    'Close',
    'Open',
    'High',
    'Low',
    'Bitcoin Total Transaction Fees USD',
    'Bitcoin USD Exchange Trade Volume',
    'Bitcoin Hash Rate',
    'Bitcoin Cost Per Transaction',
    'Bitcoin Number of Transactions',
    'Crude Oil',
    'Gold',
    'Silver',
    'Coffee',
    'Heating Oil',
    'Natural Gas',
    'Platinum',
    'Palladium',
    'Copper',
    'Cocoa',
    'Sugar',
    'Oat',
    'NASDAQ Future',
    'DAX Index',
    'KOSPI Index',
    'S&P500 Future'
)

data_with_wma = calc_wma(data_with_lags, wma_features, 30)

In [7]:
data = data_with_wma.dropna().copy()
to_drop = list(set(lag_features + wma_features))
to_drop.remove('Open')
to_drop.remove('Direction')
data.drop(to_drop, axis=1, inplace=True)

# Data Partitioning

In [8]:
target = 'Direction'

ds_int1 = data.loc['2013-08-19':'2016-07-19',:]
ds_int2 = data.loc['2013-04-01':'2017-04-01',:]

X_int1 = ds_int1.drop(target, axis=1).to_numpy()
y_int1 = ds_int1[target].to_numpy()

X_int2 = ds_int2.drop(target, axis=1).to_numpy()
y_int2 = ds_int2[target].to_numpy()

In [9]:
print(f'X_int1: {X_int1.shape}')
print(f'y_int1: {y_int1.shape}')
print(f'X_int2: {X_int2.shape}')
print(f'y_int2: {y_int2.shape}')

X_int1: (1066, 96)
y_int1: (1066,)
X_int2: (1460, 96)
y_int2: (1460,)


# Experiments Interval 1

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_int1, y_int1)
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (853, 96)
y_train: (853,)
X_test: (213, 96)
y_test: (213,)


In [11]:
ccsa_filter_selected, f_hist = CCSA_filter(X_train, y_train)
ga_filter_selected, ga = genetic_filter(X_train, y_train)

In [12]:
selected = data.drop(target, axis=1).columns[ccsa_filter_selected].values
print(f'CCSA Filter Selected ({len(selected)}):')
for s in selected:
    print(s)

CCSA Filter Selected (40):
Direction_L1
Bitcoin Total Transaction Fees USD_L1
Bitcoin USD Exchange Trade Volume_L1
Bitcoin Cost Per Transaction_L1
High_L2
Direction_L2
Bitcoin Cost Per Transaction_L2
Close_L3
Low_L3
Bitcoin USD Exchange Trade Volume_L3
High_L4
Bitcoin Hash Rate_L4
Bitcoin Number of Transactions_L4
Close_L5
Low_L5
Bitcoin USD Exchange Trade Volume_L5
Bitcoin Number of Transactions_L5
High_L6
Direction_L6
Bitcoin USD Exchange Trade Volume_L6
Bitcoin Hash Rate_L6
Bitcoin Cost Per Transaction_L6
Close_L7
Open_L7
High_L7
Direction_L7
Bitcoin Total Transaction Fees USD_L7
Bitcoin Hash Rate_L7
Bitcoin Cost Per Transaction_L7
Close_WMA30
Bitcoin Number of Transactions_WMA30
Crude Oil_WMA30
Coffee_WMA30
Natural Gas_WMA30
Platinum_WMA30
Palladium_WMA30
Copper_WMA30
Sugar_WMA30
Oat_WMA30
S&P500 Future_WMA30


In [13]:
selected = data.drop(target, axis=1).columns[ga_filter_selected].values
print(f'GA Filter Selected ({len(selected)}):')
for s in selected:
    print(s)

GA Filter Selected (29):
High_L2
Direction_L2
Bitcoin USD Exchange Trade Volume_L2
High_L3
Direction_L3
Bitcoin USD Exchange Trade Volume_L3
Open_L4
High_L4
Low_L4
Bitcoin Number of Transactions_L4
Close_L5
Direction_L5
Bitcoin USD Exchange Trade Volume_L5
Bitcoin Hash Rate_L5
Bitcoin Cost Per Transaction_L5
Close_L6
Bitcoin USD Exchange Trade Volume_L6
Direction_L7
Bitcoin USD Exchange Trade Volume_L7
Close_WMA30
Low_WMA30
Bitcoin USD Exchange Trade Volume_WMA30
Bitcoin Number of Transactions_WMA30
Crude Oil_WMA30
Gold_WMA30
Heating Oil_WMA30
Palladium_WMA30
Sugar_WMA30
KOSPI Index_WMA30


### SVM

In [14]:
results = []

pipe = make_pipeline(
    StandardScaler(),
    SVC(kernel='poly', max_iter=5e5)
)

params, _ = find_best_params(
    pipe,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/X_train.shape[1]] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    pipe,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/len(ccsa_filter_selected)] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    pipe,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/len(ga_filter_selected)] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    pipe,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    pipe,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/len(wrapper_selected)] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'svm_int1.bin')

### RandomForestClassifier

In [15]:
results = []

model = RandomForestClassifier()

params, _ = find_best_params(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train, y_train, X_test, y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    model,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    model,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'rnd_forest_int1.bin')

### ExtraTreesClassifier

In [16]:
results = []

model = ExtraTreesClassifier()

params, _ = find_best_params(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train, y_train, X_test, y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    model,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    model,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'extrees_int1.bin')

### XGBClassifier

In [17]:
results = []

model = XGBClassifier()

params, _ = find_best_params(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train, y_train, X_test, y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    model,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    model,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 1')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'xgb_int1.bin')

### MLP

In [18]:
results = []

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [20, 100, 200, 300, 400, 500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
pipe = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver='adam', activation='tanh', learning_rate='adaptive')
)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    params, _ = find_best_params(
        pipe,
        X_train,
        y_train,
        X_test,
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'All Features')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ccsa_filter_selected],
        y_train,
        X_test[:,ccsa_filter_selected],
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Filter')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ga_filter_selected],
        y_train,
        X_test[:,ga_filter_selected],
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'GA Filter')
    results.append(tests)
    
    wrapper_selected, f_wrapper_hist = CCSA_wrapper(
        X_train,
        y_train,
        pipe,
        5,
        20,
        3,
        99
    )

    params, _ = find_best_params(
        pipe,
        X_train[:,wrapper_selected],
        y_train,
        X_test[:,wrapper_selected],
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Wrapper')
    results.append(tests)
    
    dump_results(results, 'mlp_int1.bin')

### Ensemble 1

In [19]:
results = []

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [20, 100, 200, 300, 400, 500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
pipe = make_pipeline(
    StandardScaler(),
    Ensemble1()
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    params, _ = find_best_params(
        pipe,
        X_train,
        y_train,
        X_test,
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'All Features')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ccsa_filter_selected],
        y_train,
        X_test[:,ccsa_filter_selected],
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )
    
    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Filter')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ga_filter_selected],
        y_train,
        X_test[:,ga_filter_selected],
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )
    
    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'GA Filter')
    results.append(tests)
    
    wrapper_selected, f_wrapper_hist = CCSA_wrapper(
        X_train,
        y_train,
        pipe,
        5,
        20,
        3,
        99
    )

    params, _ = find_best_params(
        pipe,
        X_train[:,wrapper_selected],
        y_train,
        X_test[:,wrapper_selected],
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Wrapper')
    results.append(tests)
    
    dump_results(results, 'ensemble1_int1.bin')

### Ensemble 2

In [20]:
results = []

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [20, 100, 200, 300, 400, 500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
pipe = make_pipeline(
    StandardScaler(),
    Ensemble2()
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    params, _ = find_best_params(
        pipe,
        X_train,
        y_train,
        X_test,
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'All Features')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ccsa_filter_selected],
        y_train,
        X_test[:,ccsa_filter_selected],
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )
    
    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Filter')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ga_filter_selected],
        y_train,
        X_test[:,ga_filter_selected],
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )
    
    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'GA Filter')
    results.append(tests)
    
    wrapper_selected, f_wrapper_hist = CCSA_wrapper(
        X_train,
        y_train,
        pipe,
        5,
        20,
        3,
        99
    )

    params, _ = find_best_params(
        pipe,
        X_train[:,wrapper_selected],
        y_train,
        X_test[:,wrapper_selected],
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
    tests.insert(0, 'Interval 1')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Wrapper')
    results.append(tests)
    
    dump_results(results, 'ensemble2_int1.bin')

# Experiments Interval 2

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_int2, y_int2, test_size=.15)
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (1241, 96)
y_train: (1241,)
X_test: (219, 96)
y_test: (219,)


In [22]:
ccsa_filter_selected, f_hist = CCSA_filter(X_train, y_train)
ga_filter_selected, ga = genetic_filter(X_train, y_train)

In [23]:
selected = data.drop(target, axis=1).columns[ccsa_filter_selected].values
print(f'CCSA Filter Selected ({len(selected)}):')
for s in selected:
    print(s)

CCSA Filter Selected (52):
Low_L1
Direction_L1
Bitcoin Total Transaction Fees USD_L1
Bitcoin Cost Per Transaction_L1
High_L2
Low_L2
Direction_L2
Bitcoin Total Transaction Fees USD_L2
Bitcoin USD Exchange Trade Volume_L2
Bitcoin Hash Rate_L2
Bitcoin Cost Per Transaction_L2
Bitcoin Number of Transactions_L2
Close_L3
Open_L3
Bitcoin Number of Transactions_L3
Open_L4
High_L4
Low_L4
Direction_L4
Bitcoin Number of Transactions_L4
Close_L5
Low_L5
Bitcoin USD Exchange Trade Volume_L5
Bitcoin Cost Per Transaction_L5
Close_L6
Open_L6
High_L6
Direction_L6
Bitcoin Total Transaction Fees USD_L6
Bitcoin USD Exchange Trade Volume_L6
Bitcoin Number of Transactions_L6
Open_L7
High_L7
Low_L7
Bitcoin Total Transaction Fees USD_L7
Bitcoin USD Exchange Trade Volume_L7
Bitcoin Hash Rate_L7
Close_WMA30
Low_WMA30
Bitcoin Total Transaction Fees USD_WMA30
Bitcoin Hash Rate_WMA30
Bitcoin Number of Transactions_WMA30
Crude Oil_WMA30
Coffee_WMA30
Natural Gas_WMA30
Palladium_WMA30
Copper_WMA30
Cocoa_WMA30
Sugar_WMA

In [24]:
selected = data.drop(target, axis=1).columns[ga_filter_selected].values
print(f'GA Filter Selected ({len(selected)}):')
for s in selected:
    print(s)

GA Filter Selected (30):
Open
Bitcoin Hash Rate_L1
Close_L2
Open_L2
Direction_L2
Close_L4
Low_L4
Direction_L4
Bitcoin Total Transaction Fees USD_L4
Bitcoin USD Exchange Trade Volume_L4
Bitcoin Cost Per Transaction_L4
Direction_L5
Bitcoin USD Exchange Trade Volume_L5
Bitcoin Hash Rate_L5
Bitcoin Number of Transactions_L5
Open_L6
High_L6
Direction_L6
Bitcoin Number of Transactions_L6
Open_L7
High_L7
Bitcoin USD Exchange Trade Volume_L7
Low_WMA30
Bitcoin USD Exchange Trade Volume_WMA30
Bitcoin Cost Per Transaction_WMA30
Crude Oil_WMA30
Coffee_WMA30
Heating Oil_WMA30
NASDAQ Future_WMA30
S&P500 Future_WMA30


### SVM

In [25]:
results = []

pipe = make_pipeline(
    StandardScaler(),
    SVC(kernel='poly', max_iter=5e5)
)

params, _ = find_best_params(
    pipe,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/X_train.shape[1]] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    pipe,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/X_train.shape[1]] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    pipe,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/X_train.shape[1]] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    pipe,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    pipe,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'svc__degree': np.arange(1,5),
        'svc__gamma': [1/len(wrapper_selected)] + list(np.arange(.1, 1.1, .1)),
        'svc__C': [.5, 1, 5, 10]
    }
)

_pipe = clone(pipe).set_params(**params)

tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'svm_int2.bin')

### RandomForestClassifier

In [26]:
results = []

model = RandomForestClassifier()

params, _ = find_best_params(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train, y_train, X_test, y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    model,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    model,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'rnd_forest_int2.bin')

### ExtraTreesClassifier

In [27]:
results = []

model = ExtraTreesClassifier()

params, _ = find_best_params(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train, y_train, X_test, y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    model,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    model,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'extrees_int2.bin')

### XGBClassifier

In [28]:
results = []

model = XGBClassifier()

params, _ = find_best_params(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train, y_train, X_test, y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'All Features')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ccsa_filter_selected],
    y_train,
    X_test[:,ccsa_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Filter')
results.append(tests)

params, _ = find_best_params(
    model,
    X_train[:,ga_filter_selected],
    y_train,
    X_test[:,ga_filter_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'GA Filter')
results.append(tests)

wrapper_selected, f_wrapper_hist = CCSA_wrapper(
    X_train,
    y_train,
    model,
    5,
    20,
    3,
    99
)

params, _ = find_best_params(
    model,
    X_train[:,wrapper_selected],
    y_train,
    X_test[:,wrapper_selected],
    y_test,
    {
        'max_depth': np.arange(1, 20),
        'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]
    }
)

_model = clone(model).set_params(**params)

tests = run_tests(_model, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
tests.insert(0, 'Interval 2')
tests.insert(1, params)
tests.insert(2, 'CCSA Wrapper')
results.append(tests)

dump_results(results, 'xgb_int2.bin')

### MLP

In [29]:
results = []

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [20, 100, 200, 300, 400, 500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
pipe = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver='adam', activation='tanh', learning_rate='adaptive')
)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    params, _ = find_best_params(
        pipe,
        X_train,
        y_train,
        X_test,
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'All Features')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ccsa_filter_selected],
        y_train,
        X_test[:,ccsa_filter_selected],
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Filter')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ga_filter_selected],
        y_train,
        X_test[:,ga_filter_selected],
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'GA Filter')
    results.append(tests)
    
    wrapper_selected, f_wrapper_hist = CCSA_wrapper(
        X_train,
        y_train,
        pipe,
        5,
        20,
        3,
        99
    )

    params, _ = find_best_params(
        pipe,
        X_train[:,wrapper_selected],
        y_train,
        X_test[:,wrapper_selected],
        y_test,
        {
            'mlpclassifier__hidden_layer_sizes': combinations,
            'mlpclassifier__max_iter': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Wrapper')
    results.append(tests)

    dump_results(results, 'mlp_int2.bin')

### Ensemble 1

In [31]:
results = []

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [20, 100, 200, 300, 400, 500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
pipe = make_pipeline(
    StandardScaler(),
    Ensemble1()
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    params, _ = find_best_params(
        pipe,
        X_train,
        y_train,
        X_test,
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'All Features')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ccsa_filter_selected],
        y_train,
        X_test[:,ccsa_filter_selected],
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Filter')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ga_filter_selected],
        y_train,
        X_test[:,ga_filter_selected],
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'GA Filter')
    results.append(tests)
    
    wrapper_selected, f_wrapper_hist = CCSA_wrapper(
        X_train,
        y_train,
        pipe,
        5,
        20,
        3,
        99
    )

    params, _ = find_best_params(
        pipe,
        X_train[:,wrapper_selected],
        y_train,
        X_test[:,wrapper_selected],
        y_test,
        {
            'ensemble1__mlp_hidden_layers_sizes': combinations,
            'ensemble1__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Wrapper')
    results.append(tests)
    
    dump_results(results, 'ensemble1_int2.bin')

### Ensemble 2

In [32]:
results = []

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [20, 100, 200, 300, 400, 500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
pipe = make_pipeline(
    StandardScaler(),
    Ensemble2()
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    params, _ = find_best_params(
        pipe,
        X_train,
        y_train,
        X_test,
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train, y_train, X_test, y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'All Features')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ccsa_filter_selected],
        y_train,
        X_test[:,ccsa_filter_selected],
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ccsa_filter_selected], y_train, X_test[:,ccsa_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Filter')
    results.append(tests)
    
    params, _ = find_best_params(
        pipe,
        X_train[:,ga_filter_selected],
        y_train,
        X_test[:,ga_filter_selected],
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)
    
    tests = run_tests(_pipe, X_train[:,ga_filter_selected], y_train, X_test[:,ga_filter_selected], y_test, 50)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'GA Filter')
    results.append(tests)
    
    wrapper_selected, f_wrapper_hist = CCSA_wrapper(
        X_train,
        y_train,
        pipe,
        5,
        20,
        3,
        99
    )

    params, _ = find_best_params(
        pipe,
        X_train[:,wrapper_selected],
        y_train,
        X_test[:,wrapper_selected],
        y_test,
        {
            'ensemble2__mlp_hidden_layers_sizes': combinations,
            'ensemble2__epochs': epochs
        }
    )

    _pipe = clone(pipe).set_params(**params)

    tests = run_tests(_pipe, X_train[:,wrapper_selected], y_train, X_test[:,wrapper_selected], y_test, 1)
    tests.insert(0, 'Interval 2')
    tests.insert(1, params)
    tests.insert(2, 'CCSA Wrapper')
    results.append(tests)
    
    dump_results(results, 'ensemble2_int2.bin')

# Results Analysis

In [33]:
loaded_results = load_results()
results_table = pd.DataFrame(loaded_results, columns=['Interval', 'Params', 'Feature Selection', 'Model', 'auc_mu', 'auc_std', 'acc_mu', 'acc_std'])

In [34]:
results_table.to_excel('experiments/results_table.xlsx')

In [35]:
def extract_model_name(x):
    if isinstance(x, str):
        return x
    elif isinstance(x, Pipeline):
        return list(dict(x.named_steps).values())[-1]
    else:
        return type(x).__name__
    
results_table['Model'] = results_table['Model'].apply(extract_model_name)

In [36]:
results_table.sort_values(['Interval', 'Model', 'Feature Selection'])

,Interval,Params,Feature Selection,Model,auc_mu,auc_std,acc_mu,acc_std
36,Interval 1,"{'ensemble1__mlp_hidden_layers_sizes': (30,), ...",All Features,Ensemble1,0.51,0.03,51.13,5.16
37,Interval 1,"{'ensemble1__mlp_hidden_layers_sizes': (25, 30...",CCSA Filter,Ensemble1,0.49,0.03,48.87,5.19
39,Interval 1,"{'ensemble1__mlp_hidden_layers_sizes': (25, 35...",CCSA Wrapper,Ensemble1,0.52,0.00,55.40,0.00
38,Interval 1,"{'ensemble1__mlp_hidden_layers_sizes': (30, 30...",GA Filter,Ensemble1,0.50,0.03,50.49,5.73
12,Interval 1,"{'ensemble2__mlp_hidden_layers_sizes': (25,), ...",All Features,Ensemble2,0.51,0.02,51.56,5.34
13,Interval 1,"{'ensemble2__mlp_hidden_layers_sizes': (35, 35...",CCSA Filter,Ensemble2,0.50,0.03,50.16,5.86
15,Interval 1,"{'ensemble2__mlp_hidden_layers_sizes': (10, 15...",CCSA Wrapper,Ensemble2,0.51,0.00,46.48,0.00
14,Interval 1,"{'ensemble2__mlp_hidden_layers_sizes': (35,), ...",GA Filter,Ensemble2,0.50,0.03,50.93,5.05
24,Interval 1,"{'max_depth': 3, 'n_estimators': 30}",All Features,ExtraTreesClassifier,0.50,0.01,56.21,3.39
25,Interval 1,"{'max_depth': 2, 'n_estimators': 70}",CCSA Filter,ExtraTreesClassifier,0.50,0.01,57.70,0.50
